## 必要なライブラリのimport(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [1]:
import os
import time
import numpy as np
import rclpy
from rclpy.node import Node
from triorb_drive_interface.msg import TriorbRunPos3 as TriorbRunPos3
from triorb_drive_interface.msg import TriorbRunVel3 as TriorbRunVel3

## 事前定義(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [2]:
def get_topic_name(topic_name_raw):
    return (os.getenv('ROS_PREFIX', '')+'/'+topic_name_raw).replace("//","/")

# 相対位置制御用のトピックを配信するノード
class RelativePosNode(Node):
    def __init__(self):
        super().__init__('sample_rel_pos')
        self.pub_rel_pos = self.create_publisher(TriorbRunPos3, get_topic_name("/drive/run_pos"), 1)

    def __call__(self, x=0.0, y=0.0, deg=0.0, vxy=0.1, vw=0.5, acc=500, dec=500):
        msg = TriorbRunPos3()
        msg.position.x = x
        msg.position.y = y
        msg.position.deg = deg
        msg.speed.xy = vxy
        msg.speed.w = vw
        msg.speed.acc = acc
        msg.speed.dec = dec
        self.pub_rel_pos.publish( msg )
        raise KeyboardInterrupt

# 速度制御用のトピックを配信するノード
class RelativeVelNode(Node):
    def __init__(self):
        super().__init__('sample_rel_vel')
        self.pub_rel_vel = self.create_publisher(TriorbRunVel3, get_topic_name("/drive/run_vel"), 1)

    def __call__(self, vx=0.0, vy=0.0, vw=0.0, acc=500, dec=500):
        msg = TriorbRunVel3()
        msg.velocity.vx = vx
        msg.velocity.vy = vy
        msg.velocity.vw = vw
        msg.speed.xy = 0.0
        msg.speed.w = 0.0
        msg.speed.acc = acc
        msg.speed.dec = dec
        self.pub_rel_vel.publish( msg )
        raise KeyboardInterrupt


rclpy.init()
pos_node = RelativePosNode()
vel_node = RelativeVelNode()

def set_pos_relative(x, y, deg, vxy, vw, acc, dec):
    try:
        pos_node(x, y, deg, vxy, vw, acc, dec)
        rclpy.spin(pos_node)
    except KeyboardInterrupt:
        print("done.")

def set_vel_relative(vx, vy, vw, acc, dec):
    try:
        vel_node(vx, vy, vw, acc, dec)
        rclpy.spin(vel_node)
    except KeyboardInterrupt:
        print("done.")

## 相対位置指令
ロボットから見てx,y,w方向に移動します. 単位はx[m], y[m], deg[°], vxy[m/s], vw[rad/s], acc[ms], dec[ms] です. <br>
ロボットが移動完了前に次の指示を送ると移動途中に指示が上書きされます.

In [ ]:
# y方向（正面）に 0.5[m] 移動. 移動速度は0.1[m/s]で500[ms]で加減速が完了する
x=0.0
y=0.2
deg=0.0
vxy=0.1
vw=0.1
acc=500
dec=500

set_pos_relative(x, y, deg, vxy, vw, acc, dec)

## 速度指令
ロボットから見てvx,vy,vw方向に移動します. 単位は, vx[m/s], vy[m/s], vw[rad/s], acc[ms], dec[ms] です. <br>
<span style="color:#FF0000">減速命令を送るまで動き続けることに注意してください.</span>

In [ ]:
# x方向（右）に 0.2[m/s] で移動. 500[ms]で加減速が完了する.
vx=0.2
vy=0.0
vw=0.0
acc=500
dec=500

# 移動指示
set_vel_relative(vx, vy, vw, acc, dec)
# 1秒後待つ
time.sleep(1)
# 減速停止（各速度を0にする命令を送信）
set_vel_relative(0.0, 0.0, 0.0, acc, dec)